## Tests

Tests for email utilities. These import from the exported module.

In [ ]:
#| hide

import os
from dotenv import load_dotenv


In [ ]:
#| hide
# Load environment variables from .env file
load_dotenv()

True

In [ ]:
#| hide

# Test recipient
TEST_EMAIL = "abhisheksreesaila@finxplorer.com"
TEST_NAME = "Abhishek Sreesaila"

### Test SMTP Configuration

In [ ]:
#| hide

from fh_saas.utils_email import *


In [ ]:
#| hide

try:
    config = get_smtp_config()
    print("✓ SMTP Configuration loaded successfully")
    print(f"  Host: {config['host']}:{config['port']}")
    print(f"  From: {config['from_email']}")
    print(f"  TLS: {config['use_tls']}, SSL: {config['use_ssl']}")
except ValueError as e:
    print(f"✗ SMTP not configured: {e}")

✓ SMTP Configuration loaded successfully
  Host: smtp.azurecomm.net:587
  From: DoNotReply@finxplorer.com
  TLS: True, SSL: False


### Test Welcome Email (Test Mode)

In [ ]:
#| hide

from fh_saas.utils_email import send_welcome_email


In [ ]:
#| hide

result = send_welcome_email(
    to_email=TEST_EMAIL,
    to_name=TEST_NAME,
    user_name="Abhishek",
    tenant_name="FinXplorer Demo",
    dashboard_url="https://app.finxplorer.com/dashboard",
    test=False  # Set to False to actually send
)

print(f"\n✓ Welcome email test: {result['status']}")
print(f"  To: {result['to_email']}")


✓ Welcome email test: success
  To: abhisheksreesaila@finxplorer.com


### Test Invitation Email (Test Mode)

In [ ]:
#| hide

from fh_saas.utils_email import send_invitation_email



In [ ]:
#| hide

result = send_invitation_email(
    to_email=TEST_EMAIL,
    to_name=TEST_NAME,
    inviter_name="John Doe",
    tenant_name="FinXplorer Demo",
    invitation_url="https://app.finxplorer.com/invite/abc123",
    test=False  # Set to False to actually send
)

print(f"✓ Invitation email test: {result['status']}")
print(f"  To: {result['to_email']}")

✓ Invitation email test: success
  To: abhisheksreesaila@finxplorer.com


### Test Password Reset Email (Test Mode)

In [ ]:
#| hide

from fh_saas.utils_email import send_password_reset_email


In [ ]:
#| hide

result = send_password_reset_email(
    to_email=TEST_EMAIL,
    to_name=TEST_NAME,
    user_name="Abhishek",
    reset_url="https://app.finxplorer.com/reset-password/xyz789",
    test=False  # Set to False to actually send
)

print(f"✓ Password reset email test: {result['status']}")
print(f"  To: {result['to_email']}")

✓ Password reset email test: success
  To: abhisheksreesaila@finxplorer.com


In [ ]:
### Example: Built-in Template Preview

In [ ]:
#| eval: false
#| hide
# Preview built-in welcome template
from pathlib import Path
templates_dir = Path('../fh_saas/templates')
print("=== welcome.md ===")
print(templates_dir.joinpath('welcome.md').read_text())
print("\n=== invitation.md ===")
print(templates_dir.joinpath('invitation.md').read_text())
print("\n=== password_reset.md ===")
print(templates_dir.joinpath('password_reset.md').read_text())

=== welcome.md ===
## Welcome to {tenant_name}!

Hi **{user_name}**,

Your account has been successfully created. We're excited to have you on board!

You can access your dashboard using the link below:

[Go to Dashboard]({dashboard_url})

If you have any questions, feel free to reach out to our support team.

Best regards,  
The {tenant_name} Team

---

*This email was sent to {to_email}*


=== invitation.md ===
## You're Invited!

Hi there,

**{inviter_name}** has invited you to join **{tenant_name}**.

Click the link below to accept the invitation and create your account:

[Accept Invitation]({invitation_url})

**Important:** This invitation link will expire in 7 days.

If you weren't expecting this invitation, you can safely ignore this email.

---

*This email was sent to {to_email}*


=== password_reset.md ===
## Password Reset Request

Hi **{user_name}**,

We received a request to reset your password. Click the link below to create a new password:

[Reset Password]({reset_url})


### Example: Using Custom Template

In [ ]:
#| eval: false
#| hide
# Example: Create and use a custom template
from pathlib import Path

# Create a custom branded welcome template
custom_welcome = """## 🚀 Welcome to {tenant_name}, {user_name}!

We're thrilled to have you join our community.

### Getting Started
1. Set up your profile
2. Explore the dashboard
3. Connect with your team

[Launch Dashboard]({dashboard_url})

---
*Questions? Reply to this email or visit our help center.*
*Sent to {to_email}*
"""

# Save custom template
custom_path = Path('my_custom_welcome.md')
custom_path.write_text(custom_welcome)
print(f"Custom template created at: {custom_path}")

# Use custom template with send_welcome_email
# result = send_welcome_email(
#     to_email="user@example.com",
#     to_name="Jane Doe",
#     user_name="Jane",
#     tenant_name="Acme Corp",
#     dashboard_url="https://app.acme.com/dashboard",
#     test=True,
#     custom_template_path=custom_path
# )
# print(result)

# Cleanup
custom_path.unlink()
print("Custom template cleaned up")

Custom template created at: my_custom_welcome.md
Custom template cleaned up


### Test: Verify Template Loading

In [ ]:
#| hide
#| eval: false
# Test 1: Load built-in template
from pathlib import Path
templates_dir = Path('../fh_saas/templates')
template = load_template('welcome', templates_dir / 'welcome.md')
assert '{user_name}' in template
assert '{tenant_name}' in template
print("✅ Built-in template loads correctly")

✅ Built-in template loads correctly


In [ ]:
#| hide
#| eval: false
# # Test 2: Custom template path works
from pathlib import Path
import tempfile

# Create temp custom template
with tempfile.NamedTemporaryFile(mode='w', suffix='.md', delete=False) as f:
    f.write("# Custom\nHello {name}!")
    temp_path = Path(f.name)

try:
    content = load_template('ignored', temp_path)
    assert 'Hello {name}!' in content
    print("✅ Custom template path works")
finally:
    temp_path.unlink()

✅ Custom template path works


In [ ]:
#| hide
#| eval: false
# # Test 3: Missing template raises FileNotFoundError
try:
    get_template_path('nonexistent_template_xyz')
    assert False, "Should have raised FileNotFoundError"
except FileNotFoundError as e:
    assert 'nonexistent_template_xyz' in str(e) or 'Template not found' in str(e)
    print("✅ Missing template raises FileNotFoundError correctly")

✅ Missing template raises FileNotFoundError correctly
